In [31]:
import pandas as pd
import numpy as np
import os
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [32]:
file_path = '../프로젝트csv/구글평점웹크롤링.csv'
movies = pd.read_csv('../프로젝트csv/movie_df.csv')

In [33]:
# 웹크롤링 돌릴 영화 제목 
movie_title = movies['MOVIE_NM'].unique()
print(len(movie_title)),print(movie_title)

2579
['소울' '극장판 귀멸의 칼날: 무한열차편' '원더 우먼 1984' ... '열 개의 우물' '아가미'
 '뉴클래식 프로젝트 미안하다, 사랑한다']


(None, None)

In [40]:
# 동적 웹 크롤링으로 평점 받아오기 (google)
def google_crawling_grade(grade_non_list, file_path):
    dv = webdriver.Chrome()
    dv.get('https://www.google.co.kr/')
    time.sleep(6)
    el = dv.find_element(By.CSS_SELECTOR, 'textarea.gLFyf')
    
    # 기존 CSV 파일 읽기 (없으면 생성)
    try:
        movie = pd.read_csv(file_path)
    except FileNotFoundError:
        movie = pd.DataFrame({'MOVIE_NM': movie_title})
    
    for title in grade_non_list :
        el.clear()
        el.send_keys('영화 {} 평점'.format(title))
        el.send_keys(Keys.ENTER)
        time.sleep(5)
        
        try :
            grades = dv.find_element(By.CSS_SELECTOR, 'div.xt8Uw.q8U8x')
            grade = grades.text.strip()
            grade = round(float(grade), 2)
        except :
            grade = np.nan
            
        movie.loc[movie['MOVIE_NM']==title, '구글_평점'] = grade
        
        el = dv.find_element(By.CSS_SELECTOR, 'textarea.gLFyf')
    
    dv.close()
    movie.to_csv(file_path, index=False, encoding='utf-8')
    print(f"구글 평점 업데이트 완료! {file_path}에 저장되었습니다.")

In [54]:
import time
import random
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def google_crawling_grade(grade_non_list, file_path):
    # ChromeDriver 자동 다운로드 및 설정
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")

    dv = webdriver.Chrome(service=service, options=options)
    dv.get('https://www.google.co.kr/')
    time.sleep(random.uniform(3, 5))  # 초기 대기 시간 랜덤화

    el = dv.find_element(By.CSS_SELECTOR, 'textarea.gLFyf')

    # 기존 CSV 파일 읽기 (없으면 생성)
    try:
        movie = pd.read_csv(file_path)
    except FileNotFoundError:
        movie = pd.DataFrame({'MOVIE_NM': grade_non_list})

    for title in grade_non_list:
        el.clear()
        
        # 타이핑 지연으로 인간처럼 보이게 하기
        for char in f'영화 {title} 평점':
            el.send_keys(char)
            time.sleep(random.uniform(0.1, 0.3))  # 랜덤 타이핑 지연

        el.send_keys(Keys.ENTER)
        time.sleep(random.uniform(4, 6))  # 검색 후 대기 시간 랜덤화

        try:
            grades = dv.find_element(By.CSS_SELECTOR, 'div.xt8Uw.q8U8x')
            grade = grades.text.strip()
            grade = round(float(grade), 2)
        except Exception as e:
            grade = np.nan

        # 결과를 데이터프레임에 저장
        movie.loc[movie['MOVIE_NM'] == title, '구글_평점'] = grade

        # 다시 검색창으로 이동
        el = dv.find_element(By.CSS_SELECTOR, 'textarea.gLFyf')

    dv.close()

    # 결과 CSV 파일로 저장
    movie.to_csv(file_path, index=False, encoding='utf-8')
    print(f"구글 평점 업데이트 완료! {file_path}에 저장되었습니다.")

In [55]:
google_crawling_grade(movie_title, file_path)
movie = pd.read_csv(file_path)
movie.sample(50)

구글 평점 업데이트 완료! ../프로젝트csv/구글평점웹크롤링.csv에 저장되었습니다.


,MOVIE_NM,구글_평점
78,겨울밤에,NaN
2254,우당탕탕 운동회,NaN
874,"천년의 노래, 리버스",NaN
1694,사슴의 왕,NaN
2382,쿵쿵따 탐험대,NaN
1107,카사네 : 빼앗는 얼굴,NaN
1627,더 벙커,4.2
674,무림신검,NaN
80,찬실이는 복도 많지,NaN
1300,로보카폴리 안전교실 교통편,NaN
